## Scrape the Location Counts

### 1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:
* Starbucks

* Dunkin’ Donuts

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [15]:
# starbucks
response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")
starbucks_soup = BeautifulSoup(response.content, "html.parser")
starbucks_tables = starbucks_soup.find_all("table", class_ = "wpr-table")[0]


In [46]:
starbucks_rows = []
states_list = []

for th in starbucks_tables.find_all("th")[4:]:
    states = th.get_text(strip = True)
    states_list.append(states)

for i, tr in enumerate(starbucks_tables.find_all("tr")[1:]):
    state = states_list[i]

    cells = tr.find_all("td")
    
    #stores2023_tag = cells[0].find("a") or cells[0]
    #stores2023 = stores2023_tag.get_text(strip = True)

    #stores2021_tag = cells[0].find("a") or cells[1]
    #stores2021 = stores2021_tag.get_text(strip = True)

    #stores2024_tag = cells[0].find("a") or cells[2]
    #stores2024 = stores2024_tag.get_text(strip = True)
    stores2023 = cells[0].get_text(strip=True)
    stores2021 = cells[1].get_text(strip=True)
    stores2024 = cells[2].get_text(strip=True)

    # Append this data.
    #starbucks_rows.append({
        #"state": state,
        #"2023 Stores": stores2023,
        #"2021 Stores": stores2021,
        #"2024 Stores": stores2024
    #})
#for i, state in enumerate(states_list):
    starbucks_rows.append({
        "State": state,
        "Year": 2023,
        "Starbucks Stores": stores2023, 
        "Company": "Starbucks"
    })
    starbucks_rows.append({
        "State": state,
        "Year": 2021,
        "Starbucks Stores": stores2021,
        "Company": "Starbucks"
    })
    starbucks_rows.append({
        "State": state,
        "Year": 2024,
        "Starbucks Stores": stores2024, 
        "Company": "Starbucks"
    })

print(starbucks_rows)
df = pd.DataFrame(starbucks_rows)
print(df)

[{'State': 'California', 'Year': 2023, 'Starbucks Stores': '3,080', 'Company': 'Starbucks'}, {'State': 'California', 'Year': 2021, 'Starbucks Stores': '2,959', 'Company': 'Starbucks'}, {'State': 'California', 'Year': 2024, 'Starbucks Stores': '3,117', 'Company': 'Starbucks'}, {'State': 'Texas', 'Year': 2023, 'Starbucks Stores': '1,346', 'Company': 'Starbucks'}, {'State': 'Texas', 'Year': 2021, 'Starbucks Stores': '1,215', 'Company': 'Starbucks'}, {'State': 'Texas', 'Year': 2024, 'Starbucks Stores': '1,409', 'Company': 'Starbucks'}, {'State': 'Florida', 'Year': 2023, 'Starbucks Stores': '844', 'Company': 'Starbucks'}, {'State': 'Florida', 'Year': 2021, 'Starbucks Stores': '786', 'Company': 'Starbucks'}, {'State': 'Florida', 'Year': 2024, 'Starbucks Stores': '892', 'Company': 'Starbucks'}, {'State': 'Washington', 'Year': 2023, 'Starbucks Stores': '741', 'Company': 'Starbucks'}, {'State': 'Washington', 'Year': 2021, 'Starbucks Stores': '739', 'Company': 'Starbucks'}, {'State': 'Washington

In [19]:
#dunkin
dunkin_response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")
dunkin_soup = BeautifulSoup(dunkin_response.content, "html.parser")
dunkin_table = dunkin_soup.find("table", class_ = "wpr-table")

In [47]:
dunkin_rows = []
states_list = []

for th in dunkin_table.find_all("th")[3:]:
    states = th.get_text(strip = True)
    states_list.append(states)

for i, tr in enumerate(dunkin_table.find_all("tr")[1:]):
    state = states_list[i]

    cells = tr.find_all("td")
    
    #dunkin_stores2024_tag = cells[0].find("a") or cells[0]
    #dunkin_stores2024 = dunkin_stores2024_tag.get_text(strip = True)

    #dunkin_stores2023_tag = cells[0].find("a") or cells[1]
    #dunkin_stores2023 = dunkin_stores2023_tag.get_text(strip = True)

    dunkin_stores2024 = cells[0].get_text(strip=True)
    dunkin_stores2023 = cells[1].get_text(strip=True)


    # Append this data.
    #dunkin_rows.append({
        #"state": state,
        #"2024 Stores": dunkin_stores2024,
        #"2023 Stores": dunkin_stores2023
    #})
#for i, state in enumerate(states_list):
    dunkin_rows.append({
        "State": state,
        "Year": 2024,
        "Dunkin Stores": dunkin_stores2024,
        "Company": "Dunkin"
    })
    dunkin_rows.append({
        "State": state,
        "Year": 2023,
        "Dunkin Stores": dunkin_stores2023,
        "Company": "Dunkin"
    })
print(dunkin_rows)
df = pd.DataFrame(dunkin_rows)
print(df)

[{'State': 'New York', 'Year': 2024, 'Dunkin Stores': '1,431', 'Company': 'Dunkin'}, {'State': 'New York', 'Year': 2023, 'Dunkin Stores': '1,414', 'Company': 'Dunkin'}, {'State': 'Massachusetts', 'Year': 2024, 'Dunkin Stores': '1,042', 'Company': 'Dunkin'}, {'State': 'Massachusetts', 'Year': 2023, 'Dunkin Stores': '1,068', 'Company': 'Dunkin'}, {'State': 'Florida', 'Year': 2024, 'Dunkin Stores': '909', 'Company': 'Dunkin'}, {'State': 'Florida', 'Year': 2023, 'Dunkin Stores': '883', 'Company': 'Dunkin'}, {'State': 'New Jersey', 'Year': 2024, 'Dunkin Stores': '872', 'Company': 'Dunkin'}, {'State': 'New Jersey', 'Year': 2023, 'Dunkin Stores': '866', 'Company': 'Dunkin'}, {'State': 'Illinois', 'Year': 2024, 'Dunkin Stores': '711', 'Company': 'Dunkin'}, {'State': 'Illinois', 'Year': 2023, 'Dunkin Stores': '692', 'Company': 'Dunkin'}, {'State': 'Pennsylvania', 'Year': 2024, 'Dunkin Stores': '650', 'Company': 'Dunkin'}, {'State': 'Pennsylvania', 'Year': 2023, 'Dunkin Stores': '629', 'Company'

### 2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

In [48]:
#dunkin_rows = []
#starbucks_rows = []

#for i, state in enumerate(states_list):
    #dunkin_rows.append({
        #"State": state,
        #"Year": 2024,
        #"Dunkin Stores": dunkin_stores2024
    #})
    #dunkin_rows.append({
        #"State": state,
        #"Year": 2023,
        #"Dunkin Stores": dunkin_stores2023
    #})



#for i, state in enumerate(states_list):
    #starbucks_rows.append({
        #"State": state,
        #"Year": 2023,
        #"Starbucks Stores": stores2023
    #})
    #starbucks_rows.append({
        #"State": state,
        #"Year": 2021,
        #"Starbucks Stores": stores2021
   # })
    #starbucks_rows.append({
        #"State": state,
        #"Year": 2024,
        #"Starbucks Stores": stores2024
   # })

df_dunkin = pd.DataFrame(dunkin_rows)
df_starbucks = pd.DataFrame(starbucks_rows)

coffee = pd.merge(df_dunkin, df_starbucks, on=["State", "Year", "Company"], how="outer")
print(coffee)


       State  Year Dunkin Stores    Company Starbucks Stores
0    Alabama  2021           NaN  Starbucks               99
1    Alabama  2023            59     Dunkin              NaN
2    Alabama  2023           NaN  Starbucks               85
3    Alabama  2024            69     Dunkin              NaN
4    Alabama  2024           NaN  Starbucks                 
..       ...   ...           ...        ...              ...
250  Wyoming  2021           NaN  Starbucks               26
251  Wyoming  2023             1     Dunkin              NaN
252  Wyoming  2023           NaN  Starbucks               23
253  Wyoming  2024             1     Dunkin              NaN
254  Wyoming  2024           NaN  Starbucks                 

[255 rows x 5 columns]
       State  Year Dunkin Stores    Company Starbucks Stores
0    Alabama  2021           NaN  Starbucks               99
1    Alabama  2023            59     Dunkin              NaN
2    Alabama  2023           NaN  Starbucks               85


## Supplemental Data

### 4. Scrape the state names and populations from this wikipedia page. Merge these data with your coffee dataset.

In [49]:
pop_response = requests.get("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")
pop_soup = BeautifulSoup(pop_response.content, "html.parser")
pop_table = pop_soup.find("table", class_ = "wikitable")

pop_rows = []

for tr in pop_table.find_all("tr")[1:]:
    cells = tr.find_all('td')
    if len(cells) >= 3:
        state = cells[2].get_text(strip = True)
        population = cells[3].get_text(strip = True)


        pop_rows.append({
        "State": state,
        "Population": population})

pop_df = pd.DataFrame(pop_rows)

coffee_new = pd.merge(coffee, pop_df, on="State", how = "outer")
print(coffee_new)


       State    Year Dunkin Stores    Company Starbucks Stores Population
0    Alabama  2021.0           NaN  Starbucks               99  5,024,279
1    Alabama  2023.0            59     Dunkin              NaN  5,024,279
2    Alabama  2023.0           NaN  Starbucks               85  5,024,279
3    Alabama  2024.0            69     Dunkin              NaN  5,024,279
4    Alabama  2024.0           NaN  Starbucks                   5,024,279
..       ...     ...           ...        ...              ...        ...
259  Wyoming  2021.0           NaN  Starbucks               26    576,851
260  Wyoming  2023.0             1     Dunkin              NaN    576,851
261  Wyoming  2023.0           NaN  Starbucks               23    576,851
262  Wyoming  2024.0             1     Dunkin              NaN    576,851
263  Wyoming  2024.0           NaN  Starbucks                     576,851

[264 rows x 6 columns]
       State    Year Dunkin Stores    Company Starbucks Stores Population
0    Alabama  

### 5. Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these). Merge these values into your big dataset. Note: these values may be repeated for each state.

In [53]:
# stock prices
# 97.31 for starbucks
# 106.48 for dunkin 
financial_data = {
    "Company": ["Starbucks", "Dunkin"],
    "Stock Price": [97.31, 106.48]
}

financial_df = pd.DataFrame(financial_data)

financial_coffee = pd.merge(coffee_new, financial_df, on = "Company", how = "outer")
financial_coffee = financial_coffee.sort_values(by = "State").reset_index(drop=True)
print(financial_coffee)

       State    Year Dunkin Stores  ... Starbucks Stores Population Stock Price
0    Alabama  2023.0            59  ...              NaN  5,024,279      106.48
1    Alabama  2024.0           NaN  ...                   5,024,279       97.31
2    Alabama  2023.0           NaN  ...               85  5,024,279       97.31
3    Alabama  2021.0           NaN  ...               99  5,024,279       97.31
4    Alabama  2024.0            69  ...              NaN  5,024,279      106.48
..       ...     ...           ...  ...              ...        ...         ...
259  Wyoming  2021.0           NaN  ...               26    576,851       97.31
260  Wyoming  2023.0           NaN  ...               23    576,851       97.31
261  Wyoming  2024.0           NaN  ...                     576,851       97.31
262  Wyoming  2023.0             1  ...              NaN    576,851      106.48
263  Wyoming  2024.0             1  ...              NaN    576,851      106.48

[264 rows x 7 columns]
       State    

### 6. Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

In [54]:
#regions for us
regions = {
    "Conneticut": "Northeast",
    "Maine": "Northeast",
    "Massachusetts": "Northeast",
    "New Hampshire": "Northeast",
    "Rhode Island": "Northeast",
    "Vermont": "Northeast",
    "New Jersey": "Northeast",
    "New York": "Northeast",
    "Pennsylvania": "Northeast",
    "Illinois": "Midwest",
    "Indiana": "Midwest",
    "Michigan": "Midwest",
    "Ohio": "Midwest",
    "Wisconsin": "Midwest",
    "Iowa": "Midwest",
    "Kansas": "Midwest",
    "Minnesota": "Midwest",
    "Missouri": "Midwest",
    "Nebraska": "Midwest",
    "North Dakota": "Midwest",
    "South Dakota": "Midwest",
    "Delaware": "South",
    "Florida": "South",
    "Georgia": "South",
    "Maryland": "South",
    "North Carolina": "South",
    "South Carolina": "South",
    "Virginia": "South",
    "Washingtion, D.C.": "South",
    "West Virginia": "South",
    "Alabama": "South",
    "Kentucky": "South",
    "Mississippi": "South",
    "Tennessee": "South",
    "Arkansas": "South",
    "Louisiana": "South",
    "Oklahoma": "South",
    "Texas": "South",
    "Arizona": "West",
    "Colorado": "West",
    "Idaho": "West",
    "Montana": "West", 
    "Nevada": "West", 
    "New Mexico": "West",
    "Utah": "West",
    "Wyoming": "West",
    "Alaska": "West",
    "California": "West",
    "Hawaii": "West",
    "Oregon": "West",
    "Washington": "West"
}

financial_coffee["Regions"] = financial_coffee["State"].map(regions)

## Analyze

### 7. Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):
* Are some of these chains more prevalent in certain states than others? Possibly despite having less stores overall? Same questions for regions instead of states.

* How does your chosen financial metric change by state and region for each chain? For example, having 5 stores in California is very different from having 5 stores in Wyoming.

* Does the distribution of each chain’s stores match population distribution, by both state/region?

* Do the financial data match what you’d expect based on the number and locations of the stores? Why or why not?

## Automate

### Convert your code for Exercises 1-3 above to a function that takes a single argument: the URL. This function should

* Scrape the information on state names and corresponding number of store locations on the webpage specified (assume the page has a table in the same form and placement as the ones you scraped above)

* Extract the name of the company from either the URL specified or the webpage (assume the URL will have the same format as the ones used above)

* Return a clean, organized and tidy dataset. Find a page other than Starbucks and Dunkin’ Donuts to test this on to confirm that it works. It’s fine if this is not related to coffee.